In [10]:
import tensorflow as tf
# import tensorflow_datasets as tfdt
from keras import layers
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from transformers import BertTokenizer
import numpy as np
from keras.layers import Dense, MultiHeadAttention, LayerNormalization, Dropout, Input, Embedding, GlobalAveragePooling1D
from keras import Sequential

In [2]:
# Read Dataset
data = pd.read_csv('./dataset/dataset.csv')

data = data.sample(frac=1).reset_index(drop=True)


In [19]:
# Split the Dataset

train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))

print('Train Size : ', train_size)
print('Val Size : ', val_size)

train_df = data[:train_size]
val_df = data[train_size:train_size+val_size]
test_df = data[train_size+val_size:]

print(f'Train Length : {len(train_df)}')
print(f'Val Length : {len(val_df)}')
print(f'Test Length : {len(test_df)}')

Train Size :  96000
Val Size :  12000
Train Length : 96000
Val Length : 12000
Test Length : 12000


In [20]:
text_col = 'description'
label_col = 'label'
class_names = ['World', 'Sports', 'Business', 'Sci/Tech']

def subtract_one_label(text, label):
  return text,label -1

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Pake model pre trained jadi langsung jadi 

# Convert dataframe (training, validation, and testing) ke dalam tensorflow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_df[text_col].values, train_df[label_col].values))
train_data = train_data.map(subtract_one_label)
print(train_data)

val_data = tf.data.Dataset.from_tensor_slices((val_df[text_col].values, val_df[label_col].values))
val_data = val_data.map(subtract_one_label)
print(val_data)

test_data = tf.data.Dataset.from_tensor_slices((test_df[text_col].values, test_df[label_col].values))
test_data = test_data.map(subtract_one_label)
print(test_data)

<MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [26]:
# Pre Processing

BUFFER_SIZE = 10000
BATCH_SIZE = 64
MAX_LENGTH = 256

def encode_text(text, label):
  encoded_text = bert_tokenizer.encode(text.numpy().decode('utf-8', add_special_tokens=True, max_length=MAX_LENGTH, truncation=True, padding='max_length'))
  return encoded_text, label

train_data = train_data.map(lambda text, label: tf.py_function(encode_text, [text, label], [tf.int64, tf.int64]))
val_data = val_data.map(lambda text, label: tf.py_function(encode_text, [text, label], [tf.int64, tf.int64]))
test_data = test_data.map(lambda text, label: tf.py_function(encode_text, [text, label], [tf.int64, tf.int64]))

# Pad dataset and batch dataset
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1], []))
val_data = val_data.padded_batch(BATCH_SIZE, padded_shapes=([-1], []))
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1], []))


<RandomDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


In [36]:
class TransformerBlock(layers.Layer):
  def __init__(self, embedding_dimension, num_heads, ff_dim, rate=0.1):
    super(TransformerBlock, self).__init__()
    self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dimension)
    self.ffn = Sequential([
      Dense(ff_dim, activation='relu'),
      Dense(embedding_dimension)
    ])
    self.layernormal1 = LayerNormalization(epsilon=0.000001)
    self.layernormal2 = LayerNormalization(epsilon=0.000001)
    self.dropout1 = Dropout(rate)
    self.dropout2 = Dropout(rate) 

  def call(self, inputs, training):
    attention_output = self.att(inputs, inputs)
    attention_output = self.dropout1(attention_output, training=training)
    out1 = self.layernormal1(inputs + attention_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    return self.layernormal2(out1 + ffn_output)
  
# Parameter untuk model

num_heads = 2
embed_dimension = 128
ff_dim = 32

VOCAB_SIZE = bert_tokenizer.vocab_size
input = Input(shape=(MAX_LENGTH,))
embedding_layer = Embedding(input_dim=VOCAB_SIZE, output_dim=embed_dimension)(input)
transformer_block = TransformerBlock(embed_dimension, num_heads, ff_dim)
x = transformer_block(embedding_layer)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(4, activation='softmax')(x) # 4 masukin ke number of class

In [37]:



model = Model(inputs=input, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
# Training Model
# print(train_data)
# print(val_data)
history = model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: decode() takes at most 2 arguments (5 given)
Traceback (most recent call last):

  File "C:\Users\tinti\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "C:\Users\tinti\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "C:\Users\tinti\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "C:\Users\tinti\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\tinti\AppData\Local\Temp\ipykernel_8528\2539009883.py", line 8, in encode_text
    encoded_text = bert_tokenizer.encode(text.numpy().dec

In [ ]:
# Model Evaluation
test_loss, test_accuracy = model.evaluate(test_data)
print(f'Accuracy : {test_accuracy}')

In [ ]:
# Predicted
def predict(model, sentence, tokenizer, max_length, class_names):
  sample = tokenizer.encode(sentence[0], add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length')
  sample = np.array([sample]) # Convert sample to 2d Array
  predictions = model.predict(sample)
  print('Predictions : ', predictions)
  predicted_class = np.argmax(predictions[0])
  print(f'Predicted Class : {predicted_class}')
  print(f'Predicted Class Name : {class_names[predicted_class]}')

sample = ['the Red Sox last night appeared poised to move on without him as Curtis Leskanic inched closer to rejoining the club. The Sox initially indicated they would make an announcement last night on Williamson\'s injury but changed their position during the first inning of the game ...']
predict(model, sample, bert_tokenizer, MAX_LENGTH, class_names)


In [ ]:
# print(list(history.history.keys()))

# Sub plot to see models quality

plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], 'o-', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'x-', label="Validation Accuracy")
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'o-', label ='Training Loss')
plt.plot(history.history['val_loss'],'x-', label ='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Semakin turun angkanya semakin bagus